# Use Data Loader to Get Sample, Apply Selection and Make RSE List 

##### Set the run and sample you wish to look at

In [7]:
import sys
sys.path.append("old_notebooks/")
Run = 1
SampleType = "mc" # Choose from bnb, ext, nu, nue, drt
SampleCategory="runs"
Variation=""

##### Import the data loader

In [8]:
# Import the data loader
%load_ext autoreload
%autoreload 2
import data_loading as dl
from importlib import reload
reload(dl)
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### Load the sample with the new data loader

In [9]:
reload(dl)
df_bnb = dl.load_sample(Run,SampleCategory, SampleType, variation=Variation, loadpi0variables=True, loadshowervariables=True, loadrecoveryvars=True,loadsystematics=True)

Loaded rundict
data_path= /uboone/data/users/cthorpe/PELEE/run1/nuepresel/prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run1_reco2_reco2.root


##### Configure the path to the file we're looking at

In [10]:
path = dl.get_path(run_number=Run,category=SampleCategory,dataset=SampleType)
filename = dl.get_filename(run_number=Run,category=SampleCategory,dataset=SampleType)
print("path=",path)
print("filename=",filename)

path= /uboone/data/users/cthorpe/PELEE/run1/nuepresel/
filename= prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run1_reco2_reco2.root


##### Import the selection algorithms and plotter objects

In [11]:
from unblinding_far_sideband import *
import localSettings as ls

##### Prepare the selection query

In [15]:
# choose sideband, preselection, and selection
# find defintions in unblinding_far_sideband.py
sdb_key = 'None' #'None'
pre_key = 'ZP'
sel_key = 'ZPBDT'
#sdb_key = 'TwoPShr'
#pre_key = 'NP'
#sel_key = 'NPBDTAllShr' #'None' 'NPVLAllShr' 'NPLAllShr' 'NPBDTAllShr' 'NPTAllShr'
run = 1

sideband = sideband_categories[sdb_key]
preselection = preselection_categories[pre_key]
sel =  selection_categories[sel_key]

DETSYSPATH = ls.ntuple_path+'/detsys/{}_{}_{}/'.format(sideband['dir'], preselection['dir'], sel['dir'])
DETSYS = {'mc':False,'nue':False,'ccpi0':False,'ncpi0':False,'dirt':False,\
          'ncnopi':False,'nccpi':False,'ncpi0':False,'ccpi0':False}


QUERY = ""
for query in [sideband['query'],preselection['query'],sel['query']]:
    if query is None: continue
    if QUERY: QUERY += ' and '
    QUERY += query
    if run in [1, 2, 3, 12]:
        QUERY += f' and run{run}==True'

#print("Selection Used:",sdb_key+"_"+pre_key+"_"+sel_key)

##### Write the rse list of selected events to the file

In [17]:
# Format the name of the output file
selection=sdb_key + "_" + pre_key + "_" + sel_key

# Make a subdirectory of the one containing the samples
os.system("mkdir -p " + path + "/" + selection)

# Format the name of the new file
new_filename = path + "/" + selection + "/rse_" + filename + ".txt"
print("Output file is:",new_filename)

# Write the rse list to the output file
df_bnb_sel = df_bnb.query(QUERY,engine="python")
outfile = open(new_filename,'w')
outfile.write(df_bnb_sel[['run','sub','evt','reco_e']].to_string(index=False,header=False))
outfile.close()

Output file is: /uboone/data/users/cthorpe/PELEE/run1/nuepresel//None_ZP_ZPBDT/rse_prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run1_reco2_reco2.root.txt
